# Python + SQL Tech Session

Today we'll be covering:
1. Connecting to the database from python
1. Using templated SQL in python
1. getting data into and out of postgres efficiently
1. Advanced SQL
    - CTEs (WITH clauses)
    - window functions
    - indices / check plan
    - temp tables

### Some initial setup
Downloading the materials we'll need:
1. SSH to the class server
1. Make sure you're in your home directory: `cd ~`
1. Download the notebook: `wget https://raw.githubusercontent.com/dssg/mlforpublicpolicylab/master/techhelp/python_sql_tech_session.ipynb`
1. Download the sql template example: `wget https://raw.githubusercontent.com/dssg/mlforpublicpolicylab/master/techhelp/tech_session_template.sql`
1. Take a look at the sql template: `less tech_session_template.sql` (Type `q` to exit)

Install some packages in your group virtualenv (only one person should need to do this):
1. SSH to the class server (if you're not already there)
1. Activate virtualenv: `source /data/groups/{your_group}/dssg_env/bin/activate`
1. Install pandas and matplotlib: `pip install pandas matplotlib`
1. Install pyscopg2 and sqlalchemy (to connect to postgres): `pip install psycopg2-binary sqlalchemy`
1. Install ohio (tool for moving data to/from postgres): `pip install ohio`
1. Install PyYAML (to read YAML format): `pip install PyYAML`

Create a secrets file:
1. SSH to the class server (if you're not already there)
1. Make sure you're in your home directory: `cd ~`
1. Create the secrets file: `touch secrets.yaml`
1. Restrict access to the file: `chmod 600 secrets.yaml`
1. Edit the file: `nano secrets.yaml`
1. Fill it in with contents (remember, your password can be found in your `.pgpass` file):
    ```
    db:
      host: mlpolicylab.db.dssg.io
      port: 5432
      dbname: db_donorschoose_example
      user: {your_andrewid}
      password: {your_db_password}
    ```

Start up your jupyter server (detailed instructions [here](https://github.com/dssg/mlforpublicpolicylab/blob/master/techhelp/jupyter_setup.md)):
1. SSH to the class server (if you're not already there)
1. Start a screen session: `screen`
1. Choose a port (if you haven't already): `ss -lntu` (pick a port between 1024 and 65535 that is NOT on that list)
1. Make sure you're in your home directory: `cd ~`
1. Activate virtualenv: `source /data/groups/{your_group}/dssg_env/bin/activate`
1. Start your server: `jupyter notebook --port {port_from_above} --no-browser` (make note of the token here)
1. ON YOUR LOCAL MACHINE, create an SSH tunnel: `ssh -N -L localhost:8888:localhost:{YOUR_PORT} {YOUR_ANDREW_ID}@mlpolicylab.dssg.io` (or [using PuTTY on windows](https://docs.bitnami.com/bch/faq/get-started/access-ssh-tunnel/))
1. ON YOUR LOCAL MACHINE, open a browser and navigate to: `http://localhost:8888/`
1. Fill in the token from the jupyter server
1. Open this notebook
1. **Be sure to choose your group kernel from the "Kernel" menu**


## Import packages

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
import yaml

import ohio.ext.pandas

## TOPIC 1: Connect to the database from python

In [ ]:
with open('secrets.yaml', 'r') as f:
    secrets = yaml.safe_load(f)

db_params = secrets['db']
engine = create_engine('postgres://{user}:{password}@{host}:{port}/{dbname}'.format(
  host=db_params['host'],
  port=db_params['port'],
  dbname=db_params['dbname'],
  user=db_params['user'],
  password=db_params['password']    
))


We're connected to a database with data from the DonorsChoose organization. It has a few useful tables:
- `public.projects` -- general information about projects
- `public.resources` -- detailed information about requested resources
- `public.essays` -- project titles and descriptions
- `public.donations` -- separate record for each donation to a project

There's also a `sketch` schema you can use to create tables in

### Simple select statement with sqlalchemy engine

In [ ]:
sql = "SELECT projectid, schoolid, resource_type FROM public.projects LIMIT 3"

result_set = engine.execute(sql)
for rec in result_set:
    print(rec)

### Pandas will give a little cleaner output

In [ ]:
sql = "SELECT projectid, schoolid, resource_type FROM public.projects LIMIT 3"

pd.read_sql(sql, engine)

## Simple Table Manipulation with sqlalchemy (we'll do something more efficient below)

Let's create a little table to track your stocks of halloween candy (fill in your andrew id below)

In [ ]:
andrew_id =  # FILL IN YOUR andrew_id HERE!
candy_table = '{}_candy'.format(andrew_id)
table_schema = 'sketch'

Execute an appropriate CREATE statement

In [ ]:
create_sql = '''CREATE TABLE IF NOT EXISTS {}.{} (
    candy_type varchar NULL,
    amount int,
    units varchar
);'''.format(table_schema, candy_table)

engine.execute(create_sql)

**IMPORTANT NOTE**: Statements that modify the state of the database will not be physically reflected until we tell the connection to commit these changes. If you went into DBeaver now, you still wouldn't see this new table!

In [ ]:
engine.execute("COMMIT")

Now let's insert a few records (again note that we have to **commit** for the records to show up):

In [ ]:
insert_sql = '''INSERT INTO {}.{}
    (candy_type, amount, units)
    VALUES(%s, %s, %s);
'''.format(table_schema, candy_table)

records_to_insert = [('snickers', 10, 'bars'), ('candy corn', 5, 'bags'), ('peanut butter cups', 15, 'cups')]

for record in records_to_insert:
    engine.execute(insert_sql, record)

engine.execute("COMMIT")

Let's look at the results:

In [ ]:
sql = "SELECT * FROM {}.{}".format(table_schema, candy_table)

pd.read_sql(sql, engine)

Clean up: drop the table and commit:

In [ ]:
drop_sql = "DROP TABLE {}.{}".format(table_schema, candy_table)

engine.execute(drop_sql)
engine.execute("COMMIT")

## TOPIC 2: Using Templated SQL

Templating SQL statements and filling them in dynamically with python can be very helpful as you're transforming data for your projects, for instance, creating features, labels, and matrices for different temporal validation splits in your data.

We've actually been doing a little bit of this already (e.g., filling in table names and insert values above), but let's look at a couple of examples in more detail with the donors choose data. Suppose we wanted to look at the sets of projects posted on a few given days:

In [ ]:
sql_template = """
SELECT projectid, resource_type, poverty_level, date_posted
FROM public.projects
WHERE date_posted = '{}'::DATE
"""

results = []
for dt in ['2014-05-01', '2014-04-15', '2014-04-01']:
    sql = sql_template.format(dt)
    results.append(pd.read_sql(sql, engine))


Do some quick checks:
1. How many result sets did we get back?
1. Look at the first few results of one of the sets, are they all on the right date?
1. How many projects were posted on each of these days?

In [ ]:
# Number of result sets

In [ ]:
# First few records of one set

In [ ]:
# Number of projects on each date

#### Some simple data visualization

We won't go into detail here, but just to provide a quick example. See the matplot (or seaborn) documentation for more plot types and examples.

In [ ]:
ix = 0
df = results[ix].groupby('resource_type')['projectid'].count().reset_index()
dt = results[ix]['date_posted'].max()

fig, ax = plt.subplots()
ax.bar('resource_type', 'projectid', data=df)
ax.set_title('Counts by resource type for %s' % dt)
ax.set_ylabel('Number of Projects')
plt.show()

### Templated SQL stored in a file

If your queries get long or complex, you might want to move them out to separate files to keep your code a bit cleaner. We've provided an example to work with in `tech_session_template.sql` -- let's read that in here.

Note that here we're just making use of basic python templating here, but if you want to use more complex logic in your templates, check out packages like [Jinja2](https://jinja.palletsprojects.com/en/2.11.x/)

In [ ]:
# Read the template file
with open('tech_session_template.sql', 'r') as f:
    sql_template = f.read()

# Look at the contents:
print(sql_template)

**Looks like we'll need a few parameters:**
- table_schema
- table_name
- state_list
- start_dt
- end_dt

Notice as well that we've explicitly encoded all of these columns by hand, but you might want to think about how you might construct the sets of columns for one-hot encoded categoricals programmatically from the data, as well as the other types of features we've discussed (like aggregations in different time windows)...

In [ ]:
table_schema = 'public'
table_name = 'projects'
state_list = ['CA', 'NY', 'PA']
start_dt = '2014-03-14'
end_dt = '2014-04-30'

sql = sql_template.format(
    table_schema=table_schema,
    table_name=table_name,
    state_list=state_list,
    start_dt=start_dt,
    end_dt=end_dt
)

# Let's take a look...
print(sql)

**Looks like the square brackets in that state list will generate an error!**

Let's try formatting it before doing the templating:

In [ ]:
def list_to_string(l, dtype='string'):
    if dtype=='string':
        return ','.join(["'%s'" % elm for elm in l])
    else:
        return ','.join(["%s" % elm for elm in l])


state_list = list_to_string(['CA', 'NY', 'PA'])

print(state_list)

In [ ]:
sql = sql_template.format(
    table_schema=table_schema,
    table_name=table_name,
    state_list=state_list,
    start_dt=start_dt,
    end_dt=end_dt
)

# Let's take a look...
print(sql)

**Looks better!** Let's try running it now...

In [ ]:
df = pd.read_sql(sql, engine)

df.head(10)

## TOPIC 3: Getting data into and out of postgres efficiently

At the command line, one very efficient way of getting data into postgres is to stream it to a `COPY` statement on `STDIN`, this might look something like:
```
cat my_file.csv | psql -h mlpolicylab.db.dssg.io {group_database} -c "COPY {schema}.{table} FROM STDIN CSV HEADER"
```
(more details in the [postgres documentation](https://www.postgresql.org/docs/11/sql-copy.html))

Similarly, you can use the `\copy` command from within `psql` itself -- you can find [documentation here](https://www.postgresql.org/docs/11/app-psql.html) (seach for "\copy").

For today, we'll focus on a package called `ohio` that provides efficient tools for moving data between postgres and python. `ohio` provides interfaces for both `pandas` dataframes and `numpy` arrays, but we'll focus on the `pandas` tools here, which are provided via `import ohio.ext.pandas` (see the [docs for the numpy examples](https://github.com/dssg/ohio#extensions-for-numpy))

Note that `ohio` is dramatically more efficient than the built-in `df.to_sql()` (see the benchmarking graph below). The pandas function tries to be agnostic about SQL flavor by inserting data row-by-row, while `ohio` uses postgres-specific copy functionality to move the data much more quickly (and with lower memory overhead as well):

![ohio benchmarking](https://raw.githubusercontent.com/dssg/ohio/0.5.0/doc/img/profile-copy-from-dataframe-to-databas-1555458507.svg?sanitize=true)

Let's try it out by re-creating our halloween candy table.

In [ ]:
andrew_id =  # FILL IN YOUR andrew_id HERE!
candy_table = '{}_candy'.format(andrew_id)
table_schema = 'sketch'

In [ ]:
create_sql = '''CREATE TABLE IF NOT EXISTS {}.{} (
    candy_type varchar NULL,
    amount int,
    units varchar
);'''.format(table_schema, candy_table)

engine.execute(create_sql)
engine.execute("COMMIT")

### Inserting data with df.pg_copy_to()

In [ ]:
df = pd.DataFrame({
    'candy_type': ['snickers', 'cookies', 'candy apples', 'peanut butter cups', 'candy corn'],
    'amount': [1,1,2,3,5],
    'units': ['bars', 'cookies', 'apples', 'cups', 'bags']
})

# The ohio package adds a `pg_copy_to` method to your dataframes...
df.pg_copy_to(candy_table, engine, schema=table_schema, index=False, if_exists='append')

### Reading data with pd.DataFrame.pg_copy_from()

We can read the data from the table we just created using `pg_copy_from`:

In [ ]:
result_df = pd.DataFrame.pg_copy_from(candy_table, engine, schema=table_schema)

result_df

Note that `pg_copy_from` can accept a query as well:

In [ ]:
sql = """
SELECT
    CASE WHEN candy_type IN ('snickers', 'cookies', 'peanut butter cups') THEN 'has chocolate' ELSE 'non-chocolate' END AS chocolate_flag,
    SUM(amount) AS total_number
FROM {}.{}
GROUP BY 1
""".format(table_schema, candy_table)

result_df = pd.DataFrame.pg_copy_from(sql, engine)

result_df

## TOPIC 4: Advanced SQL

Finally for today, we want to talk about a few more advanced SQL functions that will likely be helpful as you're starting to prepare your features and training/test matrices. We **strongly encourage** you to do as much of that data manipulation as you can in the database, as postgres is well-optimized for this sort of work. The functions here should help make that work a bit easier as well.

The idea here is to give you an overview of some of the things that are possible that you might want to explore further. You can find a more in-depth [tutorial here](https://dssg.github.io/hitchhikers-guide/curriculum/2_data_exploration_and_analysis/advanced_sql/), with links out to additional documentation as well.

### CTEs (WITH clauses)

Common table expressions (CTEs), also known as WITH clauses, are a better alternative to subqueries both in terms of code readability as well as (in some cases) performance improvements. They can allow you to break up a complex query into consituent parts, making the logic of your code a little easier to follow.

By way of example, suppose we wanted to calculate the fraction of different types of projects (based on their requested type of resource) that were fully funded in MD in January 2013. Here's how we might do that with CTEs:

In [ ]:
sql = """
WITH md_projects AS (
    SELECT *
    FROM public.projects
    WHERE school_state='MD'
        AND date_posted BETWEEN '2013-01-01'::DATE AND '2013-01-31'::DATE
)
, total_donations AS (
    SELECT p.projectid, COALESCE(SUM(d.donation_total), 0) AS total_amount
    FROM md_projects p
    LEFT JOIN public.donations d USING(projectid)
    GROUP BY 1
)
, fully_funded AS (
    SELECT p.*, td.total_amount,
        CASE WHEN td.total_amount > p.total_price_excluding_optional_support THEN 1 ELSE 0 END AS funded_flag
    FROM md_projects p
    LEFT JOIN total_donations td USING(projectid)
)
SELECT resource_type, COUNT(*) AS num_projects, AVG(funded_flag) AS frac_funded
FROM fully_funded
GROUP BY 1
ORDER BY 3 DESC
"""

pd.read_sql(sql, engine)

In [ ]:
### HANDS-ON: For all the MD projects posted in January 2013 that received any donations
###           what is the average fraction of donations coming from teachers by resource type?
###           (note: the donations table has a boolean `is_teacher_acct` column that will be useful)

sql = """

"""

pd.read_sql(sql, engine)

### Analytic (Window) Functions

Postgres provides powerful functionality for calculating complex metrics such as within-group aggregates, running averages, etc., called "window functions" (because they operate over a defined window of the data relative to a given row):
- They are similar to aggregate functions, but instead of operating on groups of rows to produce a single row, they act on rows related to the current row to produce the same amount of rows.
- There are several window functions like `row_number`, `rank`, `ntile`, `lag`, `lead`, `first_value`, `last_value`, `nth_value`.
- And you can use any aggregation functions: `sum`, `count`, `avg`, `json_agg`, `array_agg`, etc

Supposed we want to answer a couple questions:
- What fraction of all projects in MD are posted by each schoolid?
- What is the most recently posted project for each school in MD?
- Calculate a running average of the total ask amount of the 4 most recent projects at a given school (say, `schoolid='ff2695b8b7f3ade678358f6e5c621c1e'`)

In [ ]:
## HANDS-ON: Try answering those questions with SELECT, GROUP BY, HAVING, AND WHERE alone

Now let's look at how we'd answer these questions with window functions...

**Fraction of projects by school**

Here, we'll group by schools but calculate the number of projects across all schools in MD using:

`SUM(COUNT(*)) OVER ()`

In that statement, `COUNT(*)` is the number of projects at the given school, then we're summing that count across all the aggregated rows with `SUM(.) OVER ()`. There, the `OVER ()` indicates the window across which to take the sum -- in this case, an empty window (that is, `()`) indicates using all records in the table.

In [ ]:
result_df = pd.read_sql("""
SELECT schoolid, 
       COUNT(*) AS num_projects, 
       1.000*COUNT(*)/SUM(COUNT(*)) OVER () AS frac_at_school
FROM public.projects
WHERE school_state = 'MD'
GROUP BY 1
ORDER BY 3 DESC
""", engine)

result_df.head()

**Most recent project by school**

Here, we'll use `row_number` to rank the projects (without ties) within school and by posting date. Note that the window here, `(PARTITION BY schoolid ORDER BY date_posted DESC)` means: within each school id, calculate a row number ordered by the posting date in descending order (so the most recent project by a given school will have `rn=1`, the second most recent will have `rn=2`, and so on).

We do this row number calculation in a CTE, allowing us to pick out the most recent project for each school simply by looking for those with `rn=1` in a subsequent step:

In [ ]:
result_df = pd.read_sql("""
WITH school_rns AS (
    SELECT *, row_number() OVER (PARTITION BY schoolid ORDER BY date_posted DESC) AS rn
    FROM public.projects
    WHERE school_state = 'MD'
)
SELECT *
FROM school_rns
WHERE rn=1
;
""", engine)

result_df.head()

**Running average of ask from last four projects**

Here, we use postgres's functionality to restrict a window to certain rows relative to the given row. Our window is:
```
(PARTITION BY schoolid ORDER BY date_posted ASC ROWS BETWEEN 3 PRECEDING AND CURRENT ROW)
```
That is,
- `PARTITION BY schoolid`: Do the calculation among records at the same school
- `ORDER BY date_posted ASC`: Order the records by posting date (earliest first)
- `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW`: Given this ordering, calculate the average across the four most recent rows (including the current row)

In [ ]:
result_df = pd.read_sql("""
SELECT date_posted, projectid, schoolid, total_price_excluding_optional_support AS current_ask,
      AVG(total_price_excluding_optional_support) OVER (
          PARTITION BY schoolid ORDER BY date_posted ASC
          ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
      ) AS running_avg_ask
FROM public.projects
WHERE schoolid = 'ff2695b8b7f3ade678358f6e5c621c1e'
ORDER BY date_posted DESC
;
""", engine)

result_df.head(10)

**Days since last project was posted**

We can use the `lag()` window function to get the date of the most recent previously-posted project (see also `last_value` for more flexibility):

In [ ]:
result_df = pd.read_sql("""
SELECT date_posted, projectid, schoolid, total_price_excluding_optional_support AS current_ask,
      date_posted::DATE - (lag(date_posted) OVER (PARTITION BY schoolid ORDER BY date_posted ASC))::DATE AS days_since_last_proj
FROM public.projects
WHERE schoolid = 'ff2695b8b7f3ade678358f6e5c621c1e'
ORDER BY date_posted DESC
;
""", engine)

result_df.head(5)

In [ ]:
# What happens when we hit the end of the series?
result_df.tail(5)

Notice the `NaN` (will be `NULL` in postgres) for the first record that doesn't have any previously-posted project, so you'd have to think about how you wanted to handle these edge cases in your feature development.

### Indices / Checking the Query Plan

Indices are particularly critical to the performance of postgres queries, especially as the data gets larger. You should think about adding indices to tables based on columns that will frequently be used for joins or filtering rows with `WHERE` clauses.

A useful tool for understanding how the database will treat a given query is checking the query plan by using the `EXPLAIN` keyword before a `SELECT` statement:

In [ ]:
# Eliminate column width truncating
pd.set_option('display.max_colwidth', None)

In [ ]:
pd.read_sql("""
EXPLAIN SELECT * FROM public.projects WHERE projectid = '32943bb1063267de6ed19fc0ceb4b9a7'
""", engine)

Notice that picking out a specific project is making use of the index via `Index Scan`.

By contrast, if we select projects for a given school:

In [ ]:
pd.read_sql("""
EXPLAIN SELECT * FROM public.projects WHERE schoolid = 'ff2695b8b7f3ade678358f6e5c621c1e'
""", engine)

Here, `Seq Scan` tells us that postgres has to scan the entire table to find the right projects, which can be very expensive (especially with joins!). Also note how much higher the overall estimated cost is for this query in the first row here than for the query above.

Likewise for joins, compare the two query plans below:

In [ ]:
pd.read_sql("""
EXPLAIN SELECT * FROM public.projects JOIN public.donations USING(projectid)
""", engine)

In [ ]:
## NOTE: Please don't actually run this query without the select!!!

pd.read_sql("""
EXPLAIN SELECT * FROM public.projects p JOIN public.donations d ON d.donation_timestamp > p.date_posted
""", engine)

**CREATING INDICES**

When you need to create indices as you build tables for your project, you can use this syntax:

```
CREATE INDEX ON {schema}.{table}({column});
```

Note that you can also specify a list of columns. If the given column (or set of columns) is a unique key for the table, you can get additional gains by declaring it as a primary key instead of simply creating an index:

```
ALTER TABLE {schema}.{table} ADD PRIMARY KEY ({column});
```

You can also find a little more documentation of postgres indices [here](https://www.postgresqltutorial.com/postgresql-indexes/postgresql-create-index/)

### Temporary Tables

Breaking up complex queries with CTEs can make your code much more readable and may provide some performance gains, but further gains can often be realized by creating and indexing temporary tables. 

Let's rework one of the CTE examples from above using temporary tables: For all the MD projects posted in January 2013 that received any donations what is the average fraction of donations coming from teachers by resource type?

In [ ]:
andrew_id =  # FILL IN YOUR andrew_id HERE!

# Temporary table and index for projects posted by MD schools in Jan 2013
engine.execute("""
CREATE LOCAL TEMPORARY TABLE tmp_{}_md_projects
    ON COMMIT PRESERVE ROWS
    AS
    SELECT *
    FROM public.projects
    WHERE school_state='MD'
        AND date_posted BETWEEN '2013-01-01'::DATE AND '2013-01-31'::DATE
;
""".format(andrew_id))
engine.execute("""CREATE INDEX ON tmp_{}_md_projects(projectid);""".format(andrew_id))
engine.execute("COMMIT;")

# Temporary table and index for donations by teachers
engine.execute("""
CREATE LOCAL TEMPORARY TABLE tmp_{}_teacher_donations
    ON COMMIT PRESERVE ROWS
    AS
    SELECT d.projectid, SUM(CASE WHEN is_teacher_acct THEN d.donation_total ELSE 0 END)/SUM(d.donation_total) AS teacher_frac
    FROM tmp_{}_md_projects p
    JOIN public.donations d USING(projectid)
    GROUP BY 1
;
""".format(andrew_id, andrew_id))
engine.execute("""CREATE INDEX ON tmp_{}_teacher_donations(projectid);""".format(andrew_id))
engine.execute("COMMIT;")

# Join these two temporary tables to get our result
pd.read_sql("""
SELECT p.resource_type, AVG(td.teacher_frac) AS avg_teacher_frac
FROM tmp_{}_md_projects p
JOIN tmp_{}_teacher_donations td USING(projectid)
GROUP BY 1
ORDER BY 2 DESC
""".format(andrew_id, andrew_id), engine)



## Clean Up

drop the candy table and commit; dispose of the sqlalchemy engine

In [ ]:
drop_sql = "DROP TABLE {}.{}".format(table_schema, candy_table)

engine.execute(drop_sql)
engine.execute("COMMIT")

engine.execute("DROP TABLE IF EXISTS tmp_{}_md_projects".format(andrew_id))
engine.execute("COMMIT")

engine.execute("DROP TABLE IF EXISTS tmp_{}_teacher_donations".format(andrew_id))
engine.execute("COMMIT")

engine.dispose()